In [ ]:
# Brian Garten
# creating a decision tree with root node: DAMAGE_PROPERTY
# to look at what factors affect property damage the most from
# solely numeric values

# JUST MISSING ACCURACY

# Note: to extract output file, use: dot -Tpng tree_numeric.dot -o tree_numeric.png
#  Where "tree_numeric.dot" is the generated text file and "tree_numeric.png" is the desired image

import pandas as pd
from sklearn import tree
import calendar

# the node to be analyzed
root_node = 'EVENT_TYPE'
# what the depth of the leaf nodes will be
maxdepth = 2
# file name for the output ".dot" file
output_file = "tree_all_"+root_node+".dot"


# preparing and cleaning data
# select all the strictly numeric values to use
col_lst = ["BEGIN_DAY","BEGIN_TIME","END_DAY","END_TIME","STATE_FIPS","MONTH_NAME",
           "CZ_FIPS","INJURIES_DIRECT","INJURIES_INDIRECT",
           "DEATHS_DIRECT","DEATHS_INDIRECT","DAMAGE_PROPERTY","DAMAGE_CROPS",
           "BEGIN_RANGE","END_RANGE","BEGIN_LAT","BEGIN_LON","END_LAT","END_LON"] # columns to read in
# add the root node if it's not in the columns desired
if root_node not in col_lst:
    col_lst.append(root_node)
df = pd.read_csv("weather-data/StormEvents_details-ftp_v1.0_d2019_c20200416.csv", usecols=col_lst)
df = pd.concat([df, pd.read_csv("weather-data/StormEvents_details-ftp_v1.0_d2018_c20200317.csv", usecols=col_lst)])
df = pd.concat([df, pd.read_csv("weather-data/StormEvents_details-ftp_v1.0_d2017_c20200121.csv", usecols=col_lst)])
df = pd.concat([df, pd.read_csv("weather-data/StormEvents_details-ftp_v1.0_d2016_c20190817.csv", usecols=col_lst)])
df = pd.concat([df, pd.read_csv("weather-data/StormEvents_details-ftp_v1.0_d2015_c20191116.csv", usecols=col_lst)])

# restrict dataframe to a specific event type(s) - currently disabled
#dfs=[]
#for event in analyze:
#    dfs.append(df.drop(df[df.EVENT_TYPE != event].index).reset_index(drop=True))
#df = pd.concat(dfs) # comment out this line to see all events
    
# remove outside the continental US - currently disabled
#df = df.drop(df[(df.STATE == 'HAWAII') | (df.STATE == 'ALASKA') | (df.STATE == 'E PACIFIC') | (df.STATE == 'ATLANTIC NORTH') | (df.STATE == 'ATLANTIC SOUTH') | (df.STATE == 'GULF OF MEXICO') | (df.STATE == 'HAWAII WATERS') | (df.STATE == 'PUERTO RICO') | (df.STATE == 'VIRGIN ISLANDS') | (df.STATE == 'AMERICAN SAMOA')].index)

# calculate event's distance covered
df['DISTANCE'] = ((df["BEGIN_LAT"]-df["END_LAT"])**2 + (df["BEGIN_LON"]-df["END_LON"])**2)**(1/2)

# calculate event mid points, and remove the start & end locations (might want to add those back in...?)
df["MID_LAT"] = (df["BEGIN_LAT"]+df["END_LAT"]) / 2
df["MID_LON"] = (df["BEGIN_LON"]+df["END_LON"]) / 2
df = df.drop(columns=["BEGIN_LAT","END_LAT","BEGIN_LON","END_LON"])

# remove any rows with missing data, then reset the index
df = df.dropna().reset_index(drop=True)

In [18]:
# convert the non-numeric values (i.e. 10.00k) to numeric (10000)
def unit_converter(df, *argv):
    for column in argv:
        for i in range(len(df)):
            num = df.loc[i,column][:-1] # grab the number
            suffix = df.loc[i,column][-1] # grab the suffix (ie K, M, etc)
            if (suffix == 'K'): # for thousands
                num = float(num) * 1e3
            elif (suffix == 'M'): # for millions
                num = float(num) * 1e6
            elif (suffix == 'B'): # for billions
                num = float(num) * 1e9
            elif (suffix == 'T'): # for trillions - not sure if this is necessary, billions is though
                num = float(num) * 1e12
            else:
                raise ValueError(num,suffix)
            df.loc[i,column] = num
    return df

df = unit_converter(df,'DAMAGE_PROPERTY','DAMAGE_CROPS')

# convert months to numerical representation (Jan = 1, Feb = 2, etc.)
mo_to_num = {name: num for num, name in enumerate(calendar.month_name) if num}
for i in range(len(df)):
    df.loc[i,'MONTH_NAME'] = mo_to_num[df.loc[i,'MONTH_NAME']]

In [19]:
# run this in linux console to allow to run: sudo sh -c "echo 1 > /proc/sys/vm/overcommit_memory"

# separate the root from the decision 'leaves'
X = df.drop(columns=root_node)
y = df[root_node]

y = pd.get_dummies(y)

# create the decision tree
clf = tree.DecisionTreeClassifier(max_depth=maxdepth,random_state=0).fit(X, y)

In [20]:
# create the tree visual
dotstr = tree.export_graphviz(clf, feature_names=X.columns)

# remove the long list of values in the visual
dotstr = dotstr.split('\\nvalue')
string = dotstr[0]
for i in range(1,len(dotstr)):
    section = dotstr[i].split('"')
    string = string + '"' + '"'.join(section[1:])
file = open("Tree Visuals/"+output_file, "w")
file.write(string)
file.close()

In [7]:
# create the tree visual w/ values
outfile = open("Tree Visuals/"+output_file, "w")
dotstr = tree.export_graphviz(clf, out_file=outfile, feature_names=X.columns)
outfile.close()